In [12]:
import sys

def main():
    graph1 =  AdjacencyListGraph()
    graph2 =  AdjacencyMatrixGraph()


    # Test AdjacencyListGraph first
    print("AdjacencyListGraph: ")
    adj_pass = test_graph(sys.stdout, graph1)
    print()

    # Test AdjacencyMatrixGraph second
    print("AdjacencyMatrixGraph: ")
    mat_pass = test_graph(sys.stdout, graph2)

    print(f"""
Summary:
  AdjacencyListGraph:   {"PASS" if adj_pass else "FAIL"}
  AdjacencyMatrixGraph: {"PASS" if mat_pass else "FAIL"}""",
          )

def test_graph(test_feedback, graph):
    commands = (
        AddVertexCommand("A", True),
        AddVertexCommand("B", True),

        # exist, C, D, E
        GetVertexCommand("C", False),
        GetVertexCommand("A", True),
        GetVertexCommand("B", True),
        GetVertexCommand("E", False),
        GetVertexCommand("D", False),

        # vertices
        AddVertexCommand("C", True),
        AddVertexCommand("D", True),
        AddVertexCommand("E", True),

        # edges
        AddEdgeCommand("B", "C", True),
        AddEdgeCommand("C", "A", True),
        AddEdgeCommand("C", "D", True),
        AddEdgeCommand("C", "E", True),
        AddEdgeCommand("D", "C", True),
        AddEdgeCommand("E", "A", True),
        AddEdgeCommand("E", "D", True),

        # fail
        AddEdgeCommand("C", "E", False),
        AddEdgeCommand("D", "C", False),

        VerifyEdgesFromCommand("A", ()),
        VerifyEdgesFromCommand("B", ("C" )),
        VerifyEdgesFromCommand("C", ("A", "D", "E" )),
        VerifyEdgesFromCommand("D", ("C" )),
        VerifyEdgesFromCommand("E", ("A", "D" )),

        VerifyEdgesToCommand("A", ("C", "E" )),
        VerifyEdgesToCommand("B", ()),
        VerifyEdgesToCommand("C", ("B", "D" )),
        VerifyEdgesToCommand("D", ("C", "E" )),
        VerifyEdgesToCommand("E", ("C")),

        # edges
        HasEdgeCommand("A", "C", False),
        HasEdgeCommand("A", "E", False),
        HasEdgeCommand("B", "C", True),
        HasEdgeCommand("C", "A", True),
        HasEdgeCommand("C", "B", False),
        HasEdgeCommand("C", "D", True),
        HasEdgeCommand("C", "E", True),
        HasEdgeCommand("D", "C", True),
        HasEdgeCommand("E", "A", True),
        HasEdgeCommand("E", "C", False),
        HasEdgeCommand("E", "D", True)
    )

    # command, fails
    for command in commands:
        if not command.execute(test_feedback, graph):
            return False

    return True

if __name__ == '__main__':
    main()

AdjacencyListGraph: 
PASS: add_vertex("A") returned a vertex
PASS: add_vertex("B") returned a vertex
PASS: get_vertex("C") returned None
PASS: get_vertex("A") returned a vertex with a correct label
PASS: get_vertex("B") returned a vertex with a correct label
PASS: get_vertex("E") returned None
PASS: get_vertex("D") returned None
PASS: add_vertex("C") returned a vertex
PASS: add_vertex("D") returned a vertex
PASS: add_vertex("E") returned a vertex
PASS: Added edge from "B" to "C"
PASS: Added edge from "C" to "A"
PASS: Added edge from "C" to "D"
PASS: Added edge from "C" to "E"
PASS: Added edge from "D" to "C"
PASS: Added edge from "E" to "A"
PASS: Added edge from "E" to "D"
PASS: Attempt to add edge from "C" to "E" returned False
PASS: Attempt to add edge from "D" to "C" returned False
PASS: Get edges from "A"
  Actual:   []
  Expected: []
PASS: Get edges from "B"
  Actual:   [B to C]
  Expected: [B to C]
PASS: Get edges from "C"
  Actual:   [C to A, C to D, C to E]
  Expected: [C to A,

In [2]:
class Vertex:
    def __init__(self, vertex_label):
        self.vertex_label = vertex_label

    def set_label(self, new_label):
        self.vertex_label = new_label

    def get_label(self, ):
        return self.vertex_label

    # Prints self vertex's label
    def print(self, output):
        print(self.vertex_label, file=output, end='')


In [4]:
class Edge:
    def __init__(self, from_vertex, to_vertex):
        self.from_vertex = from_vertex
        self.to_vertex = to_vertex

    def equals(self, other):
        return (self.from_vertex == other.from_vertex and
                self.to_vertex == other.to_vertex)

    # Prints self edge in the form "A to B", where "A" is the from_vertex's label
    # and "B" is the to_vertex's label.
    def print(self, output):
        self.from_vertex.print(output)
        print(" to ", file=output, end='')
        self.to_vertex.print(output)


In [5]:
from abc import ABC, abstractmethod

# class for a directed, unweighted graph
class DirectedGraph:
    # Creates and adds a new vertex to the graph, provided a vertex with the
    # same label doesn't already exist in the graph. Returns the new vertex on
    # success, None on failure.
    def add_vertex(self, new_vertex_label):
        self.new_vertex_label = new_vertex_label

    # Adds a directed edge from the first to the second vertex. No change is made
    # and false is returned if the edge already exists in the graph. Otherwise the
    # new edge is added and True is returned.
    @abstractmethod
    def add_directed_edge(self, from_vertex, to_vertex):
        pass

    # Returns a list of edges with the specified from_vertex.
    @abstractmethod
    def get_edges_from(from_vertex):
        pass

    # Returns a list of edges with the specified to_vertex
    @abstractmethod
    def get_edges_to(to_vertex):
        pass

    # Returns a vertex with a matching label, or None if no such vertex exists
    @abstractmethod
    def get_vertex(self, vertex_label):
        pass

    # Returns True if this graph has an edge from from_vertex to to_vertex, False otherwise
    @abstractmethod
    def has_edge(self, from_vertex, to_vertex):
        pass

In [7]:
class AdjacencyListVertex(Vertex):
    # List of vertices adjacent to this vertex. For each vertex V in this list, V is
    # adjacent to this vertex, meaning an edge from this vertex to V exists in the graph.
    def __init__(self, label):
        super().__init__(label)
        self.adjacent = []

In [9]:
class AdjacencyListGraph(DirectedGraph):
    def __init__(self):
        self.vertices = []

    # Creates and adds a new vertex to the graph, provided a vertex with the
    # same label doesn't already exist in the graph. Returns the new vertex on
    # success, None on failure.
    def add_vertex(self, new_vertex_label):
        # Your code here (remove placeholder line below)
        if self.get_vertex(new_vertex_label) is None:
            new_vertex = AdjacencyListVertex(new_vertex_label)
            self.vertices.append(new_vertex)
            return new_vertex
    
        return None
        

    # Adds a directed edge from the first to the second vertex. If the edge
    # already exists in the graph, no change is made and False is returned.
    # Otherwise the new edge is added and True is returned.
    def add_directed_edge(self, from_vertex, to_vertex):
        # Your code here (remove placeholder line below)
        
        if self.has_edge(from_vertex, to_vertex):
            return False
        
        from_vertex.adjacent.append(to_vertex)
        return True

    # Returns a list of edges with the specified from_vertex.
    def get_edges_from(self, from_vertex):
        # Your code here (remove placeholder line below)
        edges = []
        
        for vertex in self.vertices:
            if vertex in from_vertex.adjacent:
                edges.append(Edge(from_vertex, vertex))
        return edges
        

    # Returns a list of edges with the specified to_vertex.
    def get_edges_to(self, to_vertex):
        # Your code here (remove placeholder line below)
        edges = []
        
        for vertex in self.vertices:
            if to_vertex in vertex.adjacent:
                edges.append(Edge(vertex, to_vertex))
        return edges

    # Returns a vertex with a matching label, or None if no such vertex exists
    def get_vertex(self, vertex_label):
        # Your code here (remove placeholder line below)
        for vertex in self.vertices:
            if vertex.get_label() == vertex_label:
                return vertex
        return None

    # Returns True if self graph has an edge from from_vertex to to_vertex
    def has_edge(self, from_vertex, to_vertex):
        # Your code here (remove placeholder line below)
        
        if from_vertex is None or to_vertex is None:
            return False
        
        return to_vertex in from_vertex.adjacent


In [10]:
class AdjacencyMatrixGraph(DirectedGraph):
    # If matrix_rows[X][Y] is True, then an edge exists from vertices[X] to
    # vertices[Y]
    def __init__(self):
        self.vertices = []
        self.matrix_rows = []

    # Your additional code here, if desired

    # Creates and adds a new vertex to the graph, provided a vertex with the
    # same label doesn't already exist in the graph. Returns the new vertex on
    # success, None on failure.
    def add_vertex(self, new_vertex_label):
        # Your code here (remove placeholder line below)
        if self.get_vertex(new_vertex_label) is None:
            new_vertex = Vertex(new_vertex_label)
            self.vertices.append(new_vertex)
            
            # Adding a new column
            for row in self.matrix_rows:
                row.append(False)
            
            # Adding a new row
            self.matrix_rows.append([False] * len(self.vertices))
            
            return new_vertex
        
        return None

    # Adds a directed edge from the first to the second vertex. If the edge
    # already exists in the graph, no change is made and False is returned.
    # Otherwise the new edge is added and True is returned.
    def add_directed_edge(self, from_vertex, to_vertex):
        # Your code here (remove placeholder line below)
        if self.has_edge(from_vertex, to_vertex):
            return False
        
        # getting the index of the vertices
        from_index = self.vertices.index(from_vertex)
        to_index = self.vertices.index(to_vertex)
        
        self.matrix_rows[from_index][to_index] = True
        return True

    # Returns a list of edges with the specified from_vertex
    def get_edges_from(self, from_vertex):
        # Your code here (remove placeholder line below)
        if from_vertex is None:
            return []

        from_index = self.vertices.index(from_vertex)
        edges = []
        for i, has_edge in enumerate(self.matrix_rows[from_index]):
            if has_edge:
                edges.append(Edge(from_vertex, self.vertices[i]))

        return edges

    # Returns a list of edges with the specified to_vertex
    def get_edges_to(self, to_vertex):
        # Your code here (remove placeholder line below)
        if to_vertex is None:
            return []

        to_index = self.vertices.index(to_vertex)
        edges = []
        for i, row in enumerate(self.matrix_rows):
            if row[to_index]:
                edges.append(Edge(self.vertices[i], to_vertex))

        return edges

    # Returns a vertex with a matching label, or None if no such vertex exists
    def get_vertex(self, vertex_label):
        # Your code here (remove placeholder line below)
        for vertex in self.vertices:
            if vertex.get_label() == vertex_label:
                return vertex
        return None

    # Returns True if self graph has an edge from from_vertex to to_vertex
    def has_edge(self, from_vertex, to_vertex):
        # Your code here (remove placeholder line below)
        if from_vertex is None or to_vertex is None:
            return False
        
        from_index = self.vertices.index(from_vertex)
        to_index = self.vertices.index(to_vertex)
        
        return self.matrix_rows[from_index][to_index]


In [8]:
class DirectedGraphTestCommand:
    # Returns True if the test passes, else False
    def execute(self, test_feedback, graph):
        pass

    # Utility function that checks if a particular edge is in the list of edges
    def has_edge(self, edges, from_vertex, to_vertex):
        # Iterate through the list's edges
        for edge in edges:
            if edge.from_vertex == from_vertex and edge.to_vertex == to_vertex:
                return True

        return False

    def print_edges(self,
                    output,
                    edges,
                    separator = ",",
                    prefix = "[",
                    suffix = "]"):
        # Print the prefix first
        print(prefix, file=output, end='')

        # Print edges
        if len(edges):
            edges[0].print(output)
            for edge in edges[1:]:
                print(end=separator, file=output)
                edge.print(output)

        # Print suffix string
        print(end=suffix, file=output)

class AddVertexCommand(DirectedGraphTestCommand):
    def __init__(self, vertex_label, should_succeed):
        self.vertex_label = vertex_label
        self.should_succeed = should_succeed

    def execute(self, test_feedback, graph):
        # Try to add the vertex. Return True if the addition is successful else False
        new_vertex = graph.add_vertex(self.vertex_label)

        if new_vertex:
            if not self.should_succeed:
                print(f'''FAIL: add_vertex("{self.vertex_label}") \
should have returned None due to the label already being in use''',
                      file=test_feedback)
                return False

            print(f'''PASS: add_vertex("{self.vertex_label}") returned a vertex''',
                  sep='',
                  file=test_feedback)
            return True

        if self.should_succeed:
            print(f'''FAIL: add_vertex("{self.vertex_label}") incorrectly returned None''',
                  file=test_feedback)
            return False

        print(f'''PASS: add_vertex("{self.vertex_label}") returned None because the label is already in use''',
              sep='',
              file=test_feedback)
        return True

class GetVertexCommand(DirectedGraphTestCommand):
    def __init__(self, vertex_label, vertex_should_exist):
        self.vertex_label = vertex_label
        self.vertex_should_exist = vertex_should_exist

    def execute(self, test_feedback, graph):
        # Get the vertex with the specified label
        vertex = graph.get_vertex(self.vertex_label)

        # Check if get_vertex has returned None
        if vertex:
            # If the vertex shouldn't exist, then print a failure message
            if not self.vertex_should_exist:
                print(f'''FAIL: get_vertex("{self.vertex_label}") should have returned None''',
                      file=test_feedback)
                return False

            # Verify the vertex's label
            actual_label = vertex.get_label()
            if self.vertex_label != actual_label:
                print(f'''FAIL: get_vertex("{self.vertex_label}") returned a vertex with
incorrect label "{actual_label}"''',
                      file=test_feedback)
                return False

            print(f'''PASS: get_vertex("{self.vertex_label}") returned a vertex with a correct label''',
                  sep='',
                  file=test_feedback)
            return True

        # No vertex is returned, so check if a vertex is expected
        if self.vertex_should_exist:
            print(f'''FAIL: get_vertex("{self.vertex_label}") returned None, but
should have returned a vertex''',
                  file=test_feedback)
            return False

        # PASS
        print(f'''PASS: get_vertex("{self.vertex_label}") returned None''',
              sep='',
              file=test_feedback)
        return True

class AddEdgeCommand(DirectedGraphTestCommand):
    def __init__(self, from_vertex_label, to_vertex_label, should_succeed):
        self.from_vertex_label = from_vertex_label
        self.to_vertex_label = to_vertex_label
        self.should_succeed = should_succeed

    def execute(self, test_feedback, graph):
        # Find both vertices
        from_vertex = graph.get_vertex(self.from_vertex_label)
        to_vertex = graph.get_vertex(self.to_vertex_label)

        # Add the edge
        added_edge = False
        if from_vertex and  to_vertex:
            added_edge = graph.add_directed_edge(from_vertex, to_vertex)

        if added_edge == self.should_succeed:
            # PASS
            if added_edge:
                print(f'''PASS: Added edge from "{self.from_vertex_label}" to "{self.to_vertex_label}"''',
                file=test_feedback)
            else:
                print(f'''PASS: Attempt to add edge from "{self.from_vertex_label}" to "{self.to_vertex_label}" returned False''',
                      file=test_feedback)
            return True

        # FAIL
        print(f'''FAIL: Add from "{self.from_vertex_label}" to "{self.to_vertex_label}"''',
              file=test_feedback)
        return False

class HasEdgeCommand(DirectedGraphTestCommand):
    def __init__(self, from_vertex_label, to_vertex_label, expected_return_value):
        self.from_vertex_label = from_vertex_label
        self.to_vertex_label = to_vertex_label
        self.expected_return_value = expected_return_value

    def execute(self, test_feedback, graph):
        # Find both vertices
        from_vertex = graph.get_vertex(self.from_vertex_label)
        to_vertex = graph.get_vertex(self.to_vertex_label)

        # Call has_edge() to get the actual return value
        actual = graph.has_edge(from_vertex, to_vertex)

        if actual != self.expected_return_value:
            print(f'''FAIL: has_edge() should have returned {"True " if expected else "False"}
for an edge from {from_vertex.get_label() if to_vertex else "None"}, but
instead returned {"True " if actual else "False"}''',
                  file=test_feedback)
            return False

        print(f'''PASS: has_edge() returned {"True " if self.expected_return_value else "False"} \
for an edge from {from_vertex.get_label() if from_vertex else "None"} \
to {to_vertex.get_label() if to_vertex else "None"}''',
              file=test_feedback)
        return True

class VerifyEdgesFromCommand(DirectedGraphTestCommand):
    def __init__(self, from_vertex_label, to_vertex_labels):
        self.from_vertex_label = from_vertex_label
        self.to_vertex_labels = to_vertex_labels

    def execute(self, test_feedback, graph):
        # Find from_vertex
        from_vertex = graph.get_vertex(self.from_vertex_label)
        if not from_vertex:
            print(f'''FAIL: get_vertex("{to_vertex}") returned None for \
a vertex that should exist''',
                  file=test_feedback)
            return False

        # Ask the graph for edges from from_vertex
        actual = graph.get_edges_from(from_vertex)

        passed = True
        if len(actual) == len(self.to_vertex_labels):
            for to_label in self.to_vertex_labels:
                # Get the expected to-vertex
                expected_to = graph.get_vertex(to_label)

                # If the actual list of vertices does not have the edge then the
                # test fails
                if not self.has_edge(actual, from_vertex, expected_to):
                    passed = False
                    break
        else:
            passed = False

        # Print pass or fail message along with the actual and expected collections
        print(f'''{"PASS:" if passed else "FAIL:"} Get edges from "{self.from_vertex_label}"''',
              file=test_feedback)
        self.print_edges(test_feedback, actual, ", ", "  Actual:   [", "]\n")
        print("  Expected: [", file=test_feedback, end='')
        if len(self.to_vertex_labels) > 0:
            print(f'''{self.from_vertex_label} to {self.to_vertex_labels[0]}''',
                  file=test_feedback,
                  end='')
            for to_label in self.to_vertex_labels[1:]:
                print(f''', {self.from_vertex_label} to {to_label}''',
                      file=test_feedback, end='')
        print(']', file=test_feedback)
        return passed

class VerifyEdgesToCommand(DirectedGraphTestCommand):
    def __init__(self, to_vertex_label, from_vertex_labels):
        self.to_vertex_label = to_vertex_label
        self.from_vertex_labels = from_vertex_labels

    def execute(self, test_feedback, graph):
        # Find to_vertex
        to_vertex = graph.get_vertex(self.to_vertex_label)
        if not to_vertex:
            print(f'''FAIL: get_vertex("{self.to_vertex_label}") returned None for \
a vertex that should exist''',
                  file=test_feedback)
            return False

        # Ask the graph for edges from from_vertex
        actual = graph.get_edges_to(to_vertex)

        passed = True
        if len(actual) == len(self.from_vertex_labels):
            for from_label in self.from_vertex_labels:
                # Get the expected to-vertex
                expected_from = graph.get_vertex(from_label)

                # If the actual list of vertices does not have the edge then the
                # test fails
                if not self.has_edge(actual, expected_from, to_vertex):
                    passed = False
                    break
        else:
            passed = False

        # Print pass or fail message along with actual and expected collections
        print(f'''{"PASS:" if passed else "FAIL:"} Get edges to "{self.to_vertex_label}"''',
              file=test_feedback)
        self.print_edges(test_feedback, actual, ", ", "  Actual:   [", "]\n")
        print("  Expected: [", file=test_feedback, end='')
        if len(self.from_vertex_labels) > 0:
            print(f'''{self.from_vertex_labels[0]} to {self.to_vertex_label}''',
                  file=test_feedback, end='')
            for from_label in self.from_vertex_labels[1:]:
                print(f''', {from_label} to {self.to_vertex_label}''',
                      file=test_feedback, end='')
        print(']', file=test_feedback)
        return passed